## Rich display

Standard Python allows you to override the default display of objects with a `__repr__` method:

In [2]:
class A:
    pass

A()

In [3]:
class B:
    def __repr__(self):
        return "An instance of B"

B()

An instance of B

Jupyter and IPython can also display rich output, using the capabilities of your browser.

<div class="alert alert-info">See the [Rich Output](https://github.com/ipython/ipython/blob/master/examples/IPython%20Kernel/Rich%20Output.ipynb) example notebook for more on what you can display.</div>

In [4]:
from IPython.display import Image

Image(url='http://python.org/images/python-logo.gif')

To give your own objects rich displays, define a method like `_repr_html_` or `_repr_png_`:

In [6]:
class C:
    def _repr_html_(self):
        return '<span style="font-weight: bold; color: blue;">Sea</span>'

C()

Pandas uses this to display data frames as nicely formatted tables, for instance:

In [9]:
import numpy as np
import pandas as pd

dates = pd.date_range('20130101', periods=6)
pd.DataFrame(np.random.randn(6,4), index=dates, columns=list('ABCD'))

,A,B,C,D
2013-01-01,0.819557,0.963633,-0.005421,-1.362210
2013-01-02,1.310650,1.386003,1.224179,0.300534
2013-01-03,0.726016,-1.355584,-0.293338,-0.194187
2013-01-04,-1.164461,1.130513,-1.646953,-0.044849
2013-01-05,-0.318774,-1.089818,-0.497914,-0.485423
2013-01-06,1.727198,-1.310375,-1.868830,0.376063


These are the available special methods and the values they must return:

* `_repr_html_`: return raw HTML as a string
* `_repr_json_`: return a JSONable dict
* `_repr_jpeg_`: return raw JPEG data
* `_repr_png_`: return raw PNG data
* `_repr_svg_`: return raw SVG data as a string
* `_repr_latex_`: return LaTeX commands in a string surrounded by `$`

You can define more than one format. All available formats will be computed, and the frontend selects which one to show.

All the formats are saved in the notebook file, so e.g. converting a notebook to Latex will use Latex representations where possible.

### Mimebundles

The display system uses *mimebundles*, dictionaries of representation data keyed by mimetype. They are accompanied by a dictionary of metadata.

In [8]:
from IPython.core.formatters import format_display_data
format_display_data(C())

({'text/html': '<span style="font-weight: bold; color: blue;">Sea</span>',
  'text/plain': '<__main__.C at 0x7f8736c5a208>'},
 {})

You can return mimebundles directly (e.g. to send other mimetypes) from a `_repr_mimebundle_` method.

<div class="alert alert-info">See the [Custom Display Logic](https://github.com/ipython/ipython/blob/master/examples/IPython%20Kernel/Custom%20Display%20Logic.ipynb) example notebook for more on this and other ways to customise display.</div>

## Tab completion

If IPython's tab completion is missing attributes, perhaps because they're dynamically looked up by `__getattr__`, you can help it:

In [18]:
class CompleteAttrs(object):
    def __getattr__(self, name):
        if name == 'foo':
            return 1
        if name == 'bar':
            return 2
        raise AttributeError(name)
    
    def __dir__(self):
        return ['foo', 'bar'] + object.__dir__(self)

ca = CompleteAttrs()

In [ ]:
ca.

## Magic commands

IPython provides some special commands that use shell-like syntax for convenience.

In [21]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %profile  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python  %%python

Cell magics run a whole block of code in a different way:

In [22]:
%%bash
echo $HOME

/home/takluyver


Magic functions get the text of their line/cell as input, so they can process it however they like.

We can register our own magic functions with `get_ipython().register_magic_function()`:

In [23]:
def product(line):
    p = 1
    for s in line.split():
        p *= int(s)
    return p

get_ipython().register_magic_function(product)

In [24]:
%product 5 3 9

135

In [33]:
import string

def shift_letter(c, shift):
    if c not in string.ascii_letters:
        return c
    
    z = 'z' if c in string.ascii_lowercase else 'Z'
    
    n = ord(c) + shift
    if n > ord(z):
        n -= 26
    return chr(n)

def caesar_cypher(line, cell):
    shift = 13
    if line:
        shift = int(line.strip())
    
    print(''.join(shift_letter(c, shift) for c in cell))

get_ipython().register_magic_function(caesar_cypher, magic_kind='cell')

In [35]:
%%caesar_cypher 13
This is a secret message.
It will be ROT-13 encrypted.

Guvf vf n frperg zrffntr.
Vg jvyy or EBG-13 rapelcgrq.


<div class="alert alert-info">See [Defining custom magics](http://ipython.readthedocs.io/en/stable/config/custommagics.html) in the documentation for more information on this.</div>

## IPython extensions

Custom magics are often defined in IPython extensions. These are modules which define a method named `load_ipython_extension()`.

In [37]:
!cat ipython_ext_caesar.py

import string

def shift_letter(c, shift):
    if c not in string.ascii_letters:
        return c
    
    z = 'z' if c in string.ascii_lowercase else 'Z'
    
    n = ord(c) + shift
    if n > ord(z):
        n -= 26
    return chr(n)

def caesar_cypher(line, cell):
    shift = 13
    if line:
        shift = int(line.strip())
    
    print(''.join(shift_letter(c, shift) for c in cell))

def load_ipython_extension(shell):
    shell.register_magic_function(caesar_cypher, magic_kind='cell')

You can temporarily load an extension with the `%load_ext` magic command:

In [38]:
%load_ext ipython_ext_caesar

You can also configure IPython to load extensions every time it starts.

<div class="alert alert-info">See [IPython Extensions](http://ipython.readthedocs.io/en/stable/config/extensions/index.html) for more about creating and using extensions.</div>